In [ ]:
import ee
import datetime

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-sfranze1417')


In [9]:
# -------------------------------------------------------------------------------- Inputs --------------------------------------------------------------------------------
selGeom = ee.FeatureCollection("projects/ee-sfranze1417/assets/ID15_oplande_Skjern")
nwdi_thresh = 0.05
visMax = 0.4
rwb = ['red', 'white', 'blue']

start_year = 2019
end_year = 2019

# -------------------------------------------------------------------------------- Functions --------------------------------------------------------------------------------

# Sentinel-2 cloud masking
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0)
              .And(qa.bitwiseAnd(cirrusBitMask).eq(0)))

    masked_image = image.updateMask(mask).divide(10000)
    return masked_image.set({'system:time_start': image.get('system:time_start')})


# NDWI for Sentinel-2
def calculateNDWI_s2(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)



# Function to get monthly start/end dates between two years
def generate_monthly_ranges(start_year, end_year):
    ranges = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            start = datetime.date(year, month, 1)
            if month == 12:
                end = datetime.date(year + 1, 1, 1)
            else:
                end = datetime.date(year, month + 1, 1)
            ranges.append((start, end))
    return ranges

# -------------------------------------------------------------------------------- Main --------------------------------------------------------------------------------


# Loop over monthly date ranges and export
for start_date, end_date in generate_monthly_ranges(start_year, end_year):
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterDate(str(start_date), str(end_date))
          .filterBounds(selGeom)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
          .map(maskS2clouds)
          .map(calculateNDWI_s2))

    if s2.size().getInfo() == 0:
        print(f"No images for {start_date} - {end_date}")
        continue

    # Median composite
    s2_median = s2.reduce(ee.Reducer.median(), 8)

    # NDWI threshold mask (drops everything < nwdi_thresh)
    water_mask = s2_median.select('NDWI_median').gte(nwdi_thresh)

    # Apply the mask directly so only water pixels remain
    water_only = s2_median.select('NDWI_median').updateMask(water_mask)

    # Create polygon from mask (vector footprint)
    mask_vec = water_mask.selfMask().reduceToVectors(
        geometry=selGeom,
        scale=10,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='mask',
        bestEffort=True,
        maxPixels=1e9
    ).geometry()

    # Export using mask polygon as region
    month_str = f"{start_date.year}_{start_date.month:02d}"
    ee.batch.Export.image.toDrive(
        image=water_only.select('NDWI_median'),
        description=f"S2_WaterMask_test_{month_str}",
        folder='watermasks',
        region=mask_vec,
        crs='EPSG:4326',
        fileFormat='GeoTIFF',
        scale=10,
        maxPixels=1e9
    ).start()

    print(f"Started water-only export for {month_str}")

Started water-only export for 2019_01
Started water-only export for 2019_02
Started water-only export for 2019_03
Started water-only export for 2019_04
Started water-only export for 2019_05
Started water-only export for 2019_06
Started water-only export for 2019_07
Started water-only export for 2019_08
Started water-only export for 2019_09
Started water-only export for 2019_10
Started water-only export for 2019_11
Started water-only export for 2019_12
